# Libraries for recording voice from microphone input
 
* By: Farooq Shaik (farooqfox)
* Date: February 11th 2021
* Description: Exploring basic pros and cons of voice recording libraries

# Speech Recognition

1. Records indefinitely until you pause for few seconds
2. Generates text from voice, can also save as .wav 
3. Appropriate for our use case as we just habe to tap and not hold the 'record' button?
4. Very straight forward, didnt face any issues getting it to work

In [95]:
# speechrecognition is dependant on pyAudio
# install pyAudio based on your python version from files at https://www.lfd.uci.edu/~gohlke/pythonlibs/#pyaudio
# install speechrecognition

import speech_recognition as sr
r = sr.Recognizer()
with sr.Microphone() as source:
    audio = r.listen(source)
    
    try: 
        print(r.recognize_google(audio))
        
    except Exception as e:
        print('exception:' + str(e))
        
with open("recording_sr.wav", "wb") as f:
    f.write(audio.get_wav_data())

give me a hi hat


# Sound device

1. Has both record and play capability
2. stores recording as a numpy array, can be converted to .wav too
3. can't troubleshoot an error

In [71]:
#install sounddevice
import sounddevice as sd
import soundfile as sf

# recording 
duration = 3.5  # seconds
myrecording = sd.rec(int(duration * fs), samplerate=fs, channels=2, dtype='float64')
# need to troubleshoot error : PortAudioError: Error querying device -1
sf.write('record1.wav', myrecording, sampling_frequency)

In [ ]:
# playback
filename = 'metallic-beat-short.wav'
data, fs = sf.read(filename, dtype='float64')   # Extract data and sampling rate from file
sd.play(data, fs) 
status = sd.wait()   # Wait until file is done playing

# PyAudio
1. Has both record and play capability
2. stores recording as a numpy array
3. didnt face any issues getting it to work

In [97]:
import pyaudio
import wave
 
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
CHUNK = 512
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "recordedFile.wav"
device_index = 2

# instantiate PyAudio 
audio = pyaudio.PyAudio()

''' this piece of code lets you pick your device from the list of available ones.
print("----------------------record device list---------------------")
info = audio.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')
for i in range(0, numdevices):
        if (audio.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
            print("Input Device id ", i, " - ", audio.get_device_info_by_host_api_device_index(0, i).get('name'))

print("-------------------------------------------------------------")

index = int(input())
print("recording via index "+str(index))
'''

stream = audio.open(format=FORMAT, channels=CHANNELS,
                rate=RATE, input=True,input_device_index = 1,
                frames_per_buffer=CHUNK)
print ("recording started")
Recordframes = []
 
for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    Recordframes.append(data)
print ("recording stopped")
 
stream.stop_stream()
stream.close()
audio.terminate()
 
waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
waveFile.setnchannels(CHANNELS)
waveFile.setsampwidth(audio.get_sample_size(FORMAT))
waveFile.setframerate(RATE)
waveFile.writeframes(b''.join(Recordframes))
waveFile.close()

recording started
recording stopped


In [98]:
# wf = wave.open(sys.argv[1], 'rb')
wf = wave.open("recordedFile.wav", 'rb')

# instantiate PyAudio(1)
p = pyaudio.PyAudio()

# open stream (2)
stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                channels=wf.getnchannels(),
                rate=wf.getframerate(),
                output=True)

# read data
data = wf.readframes(CHUNK)

# play stream (3)
while len(data) > 0:
    stream.write(data)
    data = wf.readframes(CHUNK)

# stop stream (4)
stream.stop_stream()
stream.close()

# close PyAudio (5)
p.terminate()